In [ ]:
import pandas as pd
from sqlalchemy import create_engine

from config import (user, password, port, address, database)


# EXTRACT

### We have three data sources that we are extracting from:
    - Fuel Stations: https://afdc.energy.gov/stations/#/analyze?fuel=ELEC&ev_levels=all 
    - Electric Vehicle registrations by state: https://afdc.energy.gov/data/10962
    - States and abbrviations: https://www.50states.com/abbreviations.htm    

### Process:
    Fuel Stations was a csv files found on above source site.
    Electric Vehicle registrations by state was a CSv files found on above source site.
    States and abbrviations was pulled from a table on the aforementioned website. It was copied into an Excel file for use.
    
    All three csv files were extracted into their own pandas dataframe.
    
    We used 2 out of 64 columns from the Fuel Stations csv; State & Fuel Type Code.
    
    We used 2 out of 2 columns from the Electric Vehicle registrations by state csv; State and Registration.  
    
    We used 2 out of 2 columns from the States and abbrviations csv file that we created; US State and Abbreviation
    
    
    

### Store Fuel Stations CSV into Dataframe

In [ ]:
csv_file = "Resources/Fuel Stations all Fuel Types USA.csv"
fuel_stations_df = pd.read_csv(csv_file,low_memory=False)
fuel_stations_df.head()

### Create new data Frame with columns

In [ ]:
station_cols = [ "State","Fuel Type Code"]
new_fuel_stations_df = fuel_stations_df[station_cols].copy()
new_fuel_stations_df.head()

# TRANSFORM

#### Fuel Stations dataframe transformations: 
        1. Rename columns 
        2. Selected for Fuel Type of Electric only
        3. Set the index for the priimary key 
        

#### State Registrations dataframe:
        1. Changed column header names

#### State Abbreviations dataframe:
        1. Changed column header names

### Transform Fuel Stations Data Frame

In [ ]:
# Rename Column Headers & Clean up Duplicates

new_fuel_stations_df = new_fuel_stations_df.rename(columns={"State": "state_abbr",
                                                           "Fuel Type Code": "fuel_type"})

new_fuel_stations_df.head()

In [ ]:
# Select all electric fuel types 
new_fuel_stations_df = new_fuel_stations_df[new_fuel_stations_df["fuel_type"]=="ELEC"]

new_fuel_stations_df.count()

In [ ]:
new_fuel_stations_df = new_fuel_stations_df.reset_index(drop=True)
new_fuel_stations_df.head()

### Transform Registrations Data Frame

In [ ]:
csv_file = "Resources/10962_ev_registration_counts_by_state.csv"
registration_df = pd.read_csv(csv_file,low_memory=False)
registration_df.dtypes

In [ ]:
registration_cols = ["State","Registration Count"]
new_registration_df= registration_df[registration_cols].copy()
new_registration_df.head()

In [ ]:
new_registration_df = new_registration_df.rename(columns={"State":"state",
                                                         "Registration Count":"registrations"})
new_registration_df.head()

### Transform State Abbreviations Data Frame

In [ ]:
csv_file = "Resources/states.csv"
states_df= pd.read_csv(csv_file, low_memory=False)
states_df.head()

In [ ]:
new_state_df = states_df.rename(columns={"US STATE":"state",
                                           "ABBREVIATION":"state_abbr"})
new_state_df.head()

# LOAD

### Steps
1. Create database connection 
2. Load DataFrames into database

 ### Create Database Connection

In [ ]:
engine = create_engine(f'postgresql://{user}:{password}@{address}:{port}/{database}')

conn = engine.connect()

In [ ]:
engine.table_names()

In [ ]:
new_fuel_stations_df.to_sql(name='fuel_stations', con = engine, if_exists='append', index=True)

In [ ]:
new_registration_df.to_sql(name='registrations', con = engine, if_exists='append', index=True)

### Join the tables


In [ ]:
new_state_df.to_sql(name='state_abbreviations', con = engine, if_exists='append', index=True)